1. 평균 색상 차이를 이용한 보정:
이 방법은 앞서 설명한 것처럼 각 Bot 이미지의 평균 색상과 Bot_0.jpg 참조 이미지의 평균 색상 사이의 차이를 계산하여 해당하는 Top 이미지의 색상을 보정합니다.

In [4]:
import sys
sys.path.append('../') 

In [34]:
# Thickness data
thickness_data = pd.read_csv('../data/Thickness.csv')
thickness_data = thickness_data.drop([33])
thickness_data.reset_index(drop=True, inplace=True)
thickness_data

,Thickness
0,48.266667
1,47.966667
2,48.166667
3,48.600000
4,59.533333
...,...
122,84.633333
123,98.366667
124,97.033333
125,96.266667


In [55]:
# Required Libraries
import cv2
import numpy as np
import pandas as pd
import os
from numpy.linalg import pinv


# 1. Average Color Difference Correction
def apply_avg_color_diff_correction(bot_img, reference_img, top_img):
    color_difference = np.mean(reference_img, axis=(0, 1)) - np.mean(bot_img, axis=(0, 1))
    corrected_img = np.clip(top_img + color_difference, 0, 255).astype(np.uint8)
    return corrected_img

# 2. KY method
def apply_ky_method(bot_img, reference_img, top_img):
    window_img = bot_img/reference_img
    transformed_img = top_img*window_img
    return transformed_img

def returnRGBaverage(img):
    R = np.mean(img[:,:,2])
    G = np.mean(img[:,:,1])
    B = np.mean(img[:,:,0])
    return np.array([R, G, B])

# Load images
img_path = '../data/box_img/'
top_img_list = [f'Top_{i}.jpg' for i in range(0, 127)]
bot_img_list = [f'Bot_{i}.jpg' for i in range(0, 127)]

top_images_data = [cv2.imread(os.path.join(img_path, 'Top', img)) for img in top_img_list]
bot_images_data = [cv2.imread(os.path.join(img_path, 'Bot', img)) for img in bot_img_list]
reference_image_data = bot_images_data[0]

transformed_images_avg = [apply_avg_color_diff_correction(bot_images_data[i], reference_image_data, top_images_data[i]) for i in range(len(top_images_data))]
transformed_images_ky = [apply_ky_method(bot_images_data[i], reference_image_data, top_images_data[i]) for i in range(len(top_images_data))]

avg_images_avg = [returnRGBaverage(img) for img in transformed_images_avg]
avg_images_ky = [returnRGBaverage(img) for img in transformed_images_ky]

avg_images_avg = pd.DataFrame(avg_images_avg, columns=['R', 'G', 'B'])
avg_images_avg = pd.concat([avg_images_avg, thickness_data], axis=1)

avg_images_ky = pd.DataFrame(avg_images_ky, columns=['R', 'G', 'B'])
avg_images_ky = pd.concat([avg_images_ky, thickness_data], axis=1)

In [58]:
avg_images_avg.to_csv('../data/avg_images_avg.csv', index=False)
avg_images_ky.to_csv('../data/avg_images_ky.csv', index=False)